# **Create dataset**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/Volumes/CAS9/Aeiwz/Documents/Thesis/Analyse/Dataset/U_noesy_pqn.csv')
meta = pd.read_excel('/Volumes/CAS9/Aeiwz/Documents/Thesis/dataset/Sample sheet for Thesis copy.xlsx', sheet_name='Urine')

In [ ]:
ppm = pd.read_csv('/Volumes/CAS9/Aeiwz/Documents/Thesis/dataset/ppm_U_noesy.csv')

In [ ]:
ppm

In [ ]:
X = df.iloc[:, 14:]
X.columns = list(np.ravel(ppm))




In [ ]:
df2 = pd.concat([meta, X], axis=1)
df2.to_csv('/Volumes/CAS9/Aeiwz/Documents/Thesis/Analyse/Dataset/U_noesy_pqn.csv', index=False)

In [ ]:
df2

In [ ]:
from glob import glob
import pandas as pd

In [ ]:
data_path = '/Volumes/CAS9/Aeiwz/Documents/Thesis/Analyse/Report/OPLSDA_P'

In [ ]:

data_path

In [ ]:
if data_path[-1] == '/':
    #remove the last /
    data_path = data_path[:-1]
    
else:
    data_path = data_path

In [ ]:
data_path

In [ ]:
from genpage import gen_page

In [ ]:
gen_page(data_path=data_path).get_files()

In [ ]:
hist_plot = glob(pathname= data_path+'/element/hist_plot/*.html')
Lingress_ = glob(pathname= data_path+'/element/Lingress/*.html')
loading_plot = glob(pathname= data_path+'/element/loading_plot/*.html')
s_plot = glob(pathname= data_path+'/element/s_plot/*.html')
score_plot = glob(pathname= data_path+'/element/score_plot/*.html')

files = pd.DataFrame({'hist_plot': hist_plot, 'Lingress_': Lingress_, 'loading_plot': loading_plot, 's_plot': s_plot, 'score_plot': score_plot})

In [ ]:
files

In [ ]:
#replace value in dataframe with ..
files = files.replace(to_replace=data_path, value='..', regex=True)

In [ ]:
files['names'] = files['hist_plot'].str.split('/').str[-1].str.split('.').str[0]
files['names'] = files['names'].str.replace('Permutation_scores_','')
files['names'] = files['names'].str.replace(' ','_')

In [ ]:
files['names'][0]

In [ ]:
files_2

In [ ]:
df = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/KOI_Xal.csv')
ppm = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/KOI_ppm.csv')
meta = pd.read_excel('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/RunOrder_CK 2.xlsx', sheet_name='Data (Original)')

In [ ]:
df.columns = list(np.ravel(ppm))

In [ ]:
df2 = pd.concat([meta, df], axis=1)

In [ ]:
df2

In [ ]:
df2.to_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/dataset_KOI_Xal.csv', index=False)

# **PCA analysis**

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/dataset_KOI_Xal.csv')
#print(df)  

In [ ]:
from lingress.unipair import Unipair

In [ ]:
df.drop(df[df['Technique'] == "QC"].index, inplace=True)

In [ ]:
df

In [ ]:
test_index = Unipair.indexing(meta = df, column_name="Cardioplegia")

In [ ]:
Scores_save

In [ ]:

# Import the required python packages including 
# the custom Chemometric Model objects
import numpy as np



from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt

from pyChemometrics.ChemometricsPCA import ChemometricsPCA
from pyChemometrics.ChemometricsScaler import ChemometricsScaler

# Use to obtain same values as in the text
np.random.seed(350)

import os
import plotly.express as px
import plotly.graph_objects as go

from sklearn import decomposition
from sklearn.preprocessing import scale
from pca_ellipse import confidence_ellipse

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score


df = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/dataset_KOI_Xal.csv')
#Drop QC samples
df = df.drop(df[df['Technique'] == 'QC'].index)

Group="Cardioplegia"

test_index = Unipair.indexing(meta = df, column_name=Group)



c_list = test_index


#Make directory
# path folder
report_path = '/Volumes/CAS9/Aeiwz/Project/KOI_CC/Report'

# Create directories if they don't exist
os.makedirs('{}/PCA_result'.format(report_path), exist_ok=True)
os.makedirs('{}/PCA_result/HTML'.format(report_path), exist_ok=True)
os.makedirs('{}/PCA_result/PNG'.format(report_path), exist_ok=True)
os.makedirs('{}/PCA_result/Scores'.format(report_path), exist_ok=True)
os.makedirs('{}/PCA_result/Loading'.format(report_path), exist_ok=True)
os.makedirs('{}/PCA_result/R2'.format(report_path), exist_ok=True)
os.makedirs('{}/PCA_result/Trajectory'.format(report_path), exist_ok=True)


for i in range(len(c_list)):
    
    

    plot_name = i
    
    plot_name = str(plot_name)

    # path folder
    PCA_result_path = '{}/PCA_result'.format(report_path)
    HTML_save = '{}/HTML'.format(PCA_result_path)
    PNG_save = '{}/PNG'.format(PCA_result_path)
    Scores_save = '{}/Scores'.format(PCA_result_path)
    Loading_save = '{}/Loading'.format(PCA_result_path)
    R2_save = '{}/R2'.format(PCA_result_path)
    Trajectory_save = '{}/Trajectory'.format(PCA_result_path)

    # Import the datasets from the /data directory
    # X for the NMR spectra and Y for the 2 outcome variables
    test_gr = df.loc[c_list[i]]

    X = test_gr.iloc[:, 112:]
    #fill nan with 0
    X = X.fillna(0)
    meta = test_gr.iloc[:, :112]
    Y = test_gr[Group]
    Y1 = pd.Categorical(Y).codes
    ppm = list(np.ravel(X.columns).astype(float))
    # Use pandas Categorical type to generate the dummy enconding of the Y vector (0 and 1) 

   

    import time

    from tqdm import tqdm

    T1 = time.time()


     # Select the scaling options: 
    # Here we are generating 3 scaling objects to explore the effect of scaling in PCA:

    # Unit-Variance (UV) scaling:
    
    scale__ = 'UV'
    scale_power_ = 1

    # Mean Centering (MC):
    #scaling_object_mc = ChemometricsScaler(scale_power=0)

    # Pareto scaling (Par):
    # scaling_object_par = ChemometricsScaler(scale_power=0.5)

    
    model_scaler = ChemometricsScaler(scale_power=scale_power_)
    model_scaler.fit(X)
    model_X = model_scaler.transform(X)

    pca_model = decomposition.PCA(n_components=2)
    pca_model.fit(model_X)

    scores_ = pca_model.transform(model_X)
    df_scores_ = pd.DataFrame(scores_, columns=['PC1', 'PC2'])
    df_scores_.index = test_gr.index

    df2_scores_ = pd.concat([df_scores_, Y], axis=1)

    #save PCA score to csv
    df2_scores_.to_csv(Scores_save+'/PCA_scores_'+ plot_name +'.csv')

    loadings_ = pca_model.components_.T
    df_loadings_ = pd.DataFrame(loadings_, columns=['PC1', 'PC2'], index=np.ravel(ppm))
    df_loadings_.to_csv(Loading_save + '/Loading_scores ' + plot_name + '.csv')

    explained_variance_ = pca_model.explained_variance_ratio_
    explained_variance_

    explained_variance_ = np.insert(explained_variance_, 0, 0)

    cumulative_variance_ = np.cumsum(np.round(explained_variance_, decimals=3))

    pc_df_ = pd.DataFrame(['','PC1', 'PC2'], columns=['PC'])
    explained_variance_df_ = pd.DataFrame(explained_variance_, columns=['Explained Variance'])
    cumulative_variance_df_ = pd.DataFrame(cumulative_variance_, columns=['Cumulative Variance'])

    df_explained_variance_ = pd.concat([pc_df_, explained_variance_df_, cumulative_variance_df_], axis=1)
    df_explained_variance_.to_csv(R2_save + '/R2 ' + plot_name + '.csv')
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=357)
    X_test = model_scaler.transform(X_test)
    X_test_pca = pca_model.transform(X_test)

    # Inverse transform the test set from the PCA space
    X_test_reconstructed = pca_model.inverse_transform(X_test_pca)


    # Calculate Q2 score for the test set
    q2_test = r2_score(X_test, X_test_reconstructed)
           

    # Plot

    # https://plotly.com/python/bar-charts/

    fig = px.bar(df_explained_variance_, 
                x='PC', y='Explained Variance',
                text='Explained Variance',
                width=800, height=600,
                title='Explained Variance ({} scaling)'.format(scale__))
    fig.update_layout(
        title={
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        font=dict(size=15))
    fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
    #fig.show()
    fig.write_image(PNG_save + "/Explained Variance " + plot_name + ".png")
    fig.write_html(HTML_save + "/Explained Variance " + plot_name + ".html")

    # https://plotly.com/python/creating-and-updating-figures/
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_explained_variance_['PC'],
            y=df_explained_variance_['Cumulative Variance'],
            marker=dict(size=15, color="LightSeaGreen"),
            name='R<sup>2</sup>X (Cum)'
        ))

    fig.add_trace(
        go.Bar(
            x=df_explained_variance_['PC'],
            y=df_explained_variance_['Explained Variance'],
            marker=dict(color="RoyalBlue"),
            name='R<sup>2</sup>X',
            text=np.round(df_explained_variance_['Explained Variance'], decimals=3)
        ))
    fig.update_layout(width=800, height=600,
                    title='Explained Variance and Cumulative Variance ' + plot_name)
    fig.update_layout(
        title={
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

    #fig.show()
    fig.write_image(PNG_save + "/Explained Variance + Cumulative Variance " + plot_name + ".png")
    fig.write_html(HTML_save + "/Explained Variance + Cumulative Variance " + plot_name + ".html")


    colour_dict = {
                    "Off pump CABG (OPCAB)": "#E91E63",        
                    "On pump CABG (ONCAB)": "#FF9800",
                    "custodiol": "#FFEB3B",       
                    "del Nido": "#9C27B0",
                    "st. thomas": "#03A9F4",
                    "No med": "#4CAF50",        
                    "7": "#B30000",
                    "8": "#3F51B5"
                    }
    

    
    

    # PCA plot
    pca_label = df2_scores_.index


    fig = px.scatter(df2_scores_, x='PC1', y='PC2', symbol=meta['Time point'],
                     
                     symbol_map={'Pre-op': 'circle',
                            'Op': 'diamond',
                            'Post-op1': 'square',
                            'Post-op2': 'triangle-up',
                            'Post-op3': 'triangle-down'},
                     
                    color=Group,
                    color_discrete_map=colour_dict, 
                    title='<b>PCA Scores Plot ({} Scaling)<b>'.format(scale__), 
                    height=900, width=1300,
                    labels={"PC1": "PC1 R<sup>2</sup>X: {} %".format(np.round(df_explained_variance_.iloc[1,1]*100, decimals=2)),
                            "PC2": "PC2 R<sup>2</sup>X: {} %".format(np.round(df_explained_variance_.iloc[2,1]*100, decimals=2))})

    #fig.add_annotation(yref = 'paper', y = -1.06, xref = 'paper', x=1.06 , text='Q2' +' = {}'.format(np.round(df_explained_variance_.iloc[2,2], decimals=2)))
    #fig.update_annotations(font = {
    #    'size': 20}, showarrow=False)

    #set data point fill alpha with boarder in each color
    fig.update_traces(marker=dict(size=35, opacity=0.7, line=dict(width=2, color='DarkSlateGrey')))

    fig.add_annotation(dict(font=dict(color="black",size=20),
                            #x=x_loc,
                            x=1.0,
                            y=0.05,
                            showarrow=False,
                            text='<b>R<sup>2</sup>X (Cum): {}%<b>'.format(np.round(df_explained_variance_.iloc[2,2]*100, decimals=2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")

    fig.add_annotation(dict(font=dict(color="black",size=20),
                            #x=x_loc,
                            x=1.0,
                            y=0.01,
                            showarrow=False,
                            text='<b>Q<sup>2</sup>X (Cum): {}%<b>'.format(np.round(q2_test*100, decimals=2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")



    fig.update_traces(marker=dict(size=20))
    #fig.update_traces(textposition='top center') #Text label position

    #fig.update_traces(marker=dict(size=12, color=Y1_color, marker=Y2_marker))
    fig.add_shape(type='path',
                path=confidence_ellipse(df2_scores_['PC1'], df2_scores_['PC2']))



    fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_layout(
        title={
            'y':1,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        font=dict(size=20))
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')

    #fig.show()
    fig.write_image(PNG_save + "/PCA " + plot_name + ".png")
    fig.write_html(HTML_save + "/PCA " + plot_name + ".html")


# Loading plot
    loadings_label = df_loadings_.index


    fig = px.line(df_loadings_, x=loadings_label, y=['PC1', 'PC2'],
                    height=600, width=1800,
                    title='Loadings ' + plot_name
                    )

    fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_layout(title={'y':0.95,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'},
                    font=dict(size=20))
    
    fig.update_layout(scene={'xaxis': {'autorange': 'reversed'}})
            
    fig.update_traces(marker=dict(size=1))
    fig.update_layout(xaxis_title="𝛿<sub>H</sub> in ppm")
    #fig.show()

    fig.write_image(PNG_save + "/Loading " + plot_name + ".png")
    fig.write_html(HTML_save + "/Loading " + plot_name + ".html")
    
    
    #Time trajectory
    traject_df = df2_scores_.copy()
    traject_df['Time point'] = meta['Time point']
    med_df = traject_df.groupby(['Time point', Group]).median()
    err_df = traject_df.groupby(['Time point', Group]).sem()
    
    med_df = med_df.reset_index()
    err_df = err_df.reset_index()
    
    
    med_df.sort_values(by=['Time point'], inplace=True, key=lambda x: x.map({"Pre-op": 0, "Op": 1, "Post-op1": 2, "Post-op2": 3, "Post-op3": 4}))
    med_df.sort_values(by=[Group], inplace=True)
    med_df.reset_index(drop=True, inplace=True)
    
    err_df.sort_values(by=['Time point'], inplace=True, key=lambda x: x.map({"Pre-op": 0, "Op": 1, "Post-op1": 2, "Post-op2": 3, "Post-op3": 4}))
    err_df.sort_values(by=[Group], inplace=True)
    err_df.reset_index(drop=True, inplace=True)
    

    list_label = med_df[Group].unique()
    #line 1
    if list_label[0] == 'Off pump CABG (OPCAB)': 
        colour1 = '#E91E63'
    elif list_label[0] == 'On pump CABG (ONCAB)':
        colour1 = '#FF9800'
    elif list_label[0] == 'custodiol':
        colour1 = '#FFEB3B'
    elif list_label[0] == 'del Nido':
        colour1 = '#9C27B0'
    elif list_label[0] == 'st. thomas':
        colour1 = '#03A9F4'
    elif list_label[0] == 'No med':
        colour1 = '#4CAF50'
    
    #line 2
    if list_label[1] == 'Off pump CABG (OPCAB)':
        colour2 = '#E91E63'
    elif list_label[1] == 'On pump CABG (ONCAB)':
        colour2 = '#FF9800'
    elif list_label[1] == 'custodiol':
        colour2 = '#FFEB3B'
    elif list_label[1] == 'del Nido':
        colour2 = '#9C27B0'
    elif list_label[1] == 'st. thomas':
        colour2 = '#03A9F4'
    elif list_label[1] == 'No med':
        colour2 = '#4CAF50'
        



    fig = px.line(med_df, x='PC1', y='PC2', line_group='Time point', error_x=err_df["PC1"], error_y=err_df["PC2"],
                    color=Group, 
                    symbol='Time point',
                    color_discrete_map=colour_dict,
                    symbol_map={'Pre-op': 'circle',
                            'Op': 'diamond',
                            'Post-op1': 'square',
                            'Post-op2': 'triangle-up',
                            'Post-op3': 'triangle-down'}, 
                    title='<b>Principle component analysis (Pareto scaling)<b>', 
                    height=900, width=1300,
                    labels={"PC1": "PC1 R<sup>2</sup>X: {} %".format(24.3),
                            "PC2": "PC2 R<sup>2</sup>X: {} %".format(15.0)})


    # create a new trace for the connecting line
    fig.add_trace(go.Scatter(
        x=med_df.loc[0:4, "PC1"], # x-coordinates of the line
        y=med_df.loc[0:4, "PC2"], # y-coordinates of the line
        mode='lines', # specify the trace type as lines
        line=dict(color=colour1, width=2), # set the color and width of the line
        showlegend=False # hide the trace from the legend
    ))


    # create a new trace for the connecting line
    fig.add_trace(go.Scatter(
        x=med_df.loc[5:9, "PC1"], # x-coordinates of the line
        y=med_df.loc[5:9, "PC2"], # y-coordinates of the line
        mode='lines', # specify the trace type as lines
        line=dict(color=colour2, width=2), # set the color and width of the line
        showlegend=False # hide the trace from the legend
    ))
    '''
    # create a new trace for the connecting line
    fig.add_trace(go.Scatter(
        x=df_score_mean.iloc[4:8, 0], # x-coordinates of the line
        y=df_score_mean.iloc[4:8, 2], # y-coordinates of the line
        mode='lines', # specify the trace type as lines
        line=dict(color='#84CC56', width=2), # set the color and width of the line
        showlegend=False # hide the trace from the legend
    ))


    # create a new trace for the connecting line
    fig.add_trace(go.Scatter(
        x=df_score_mean.iloc[8:12, 0], # x-coordinates of the line
        y=df_score_mean.iloc[8:12, 2], # y-coordinates of the line
        mode='lines', # specify the trace type as lines
        line=dict(color='#CA83CC', width=2), # set the color and width of the line
        showlegend=False # hide the trace from the legend
    ))

    # create a new trace for the connecting line
    fig.add_trace(go.Scatter(
        x=df_score_mean.iloc[8:12, 0], # x-coordinates of the line
        y=df_score_mean.iloc[8:12, 2], # y-coordinates of the line
        mode='lines', # specify the trace type as lines
        line=dict(color='#6AE022', width=2), # set the color and width of the line
        showlegend=False # hide the trace from the legend
    ))

    '''





    fig.add_annotation(dict(font=dict(color="black",size=20),
                            #x=x_loc,
                            x=1.0,
                            y=0.05,
                            showarrow=False,
                            text='<b>R<sup>2</sup>X (Cum): {}%<b>'.format(np.round(df_explained_variance_.iloc[2,2]*100, decimals=2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")

    fig.add_annotation(dict(font=dict(color="black",size=20),
                            #x=x_loc,
                            x=1.0,
                            y=0.01,
                            showarrow=False,
                            text='<b>Q<sup>2</sup>X (Cum): {}%<b>'.format(np.round(q2_test*100, decimals=2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")

    fig.update_traces(marker=dict(size=20))
    #fig.update_traces(textposition='top center') #Text label position

    #fig.update_traces(marker=dict(size=12, color=Y1_color, marker=Y2_marker))
    fig.add_shape(type='path',
                path=confidence_ellipse(med_df['PC1'],med_df['PC2']))



    #update axis as scitifics
    fig.update_xaxes(tickformat=".1e")
    fig.update_yaxes(tickformat=".1e")



    fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_layout(
        title={
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        font=dict(size=20))
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
    #fig.show()
    
    fig.write_image("{}/Save_PCA_trajectory_".format(Trajectory_save) + plot_name + ".png")
    fig.write_html("{}/Save_PCA_trajectory_".format(Trajectory_save) + plot_name + ".html")

    


    T2 = time.time()

    print('{} Done /n Time taken: {} seconds'.format(plot_name, T2-T1))


In [ ]:
df1 = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Code/dataset_1.csv')
df2 = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Code/dataset_2.csv')
df3 = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Code/dataset_3.csv')
df4 = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Code/dataset_4.csv')

In [ ]:
len(df1['Technique'].unique()), len(df2['Cardioplegia'].unique()), len(df3['Cardioplegia'].unique()), len(df4['Technique'].unique())

In [ ]:
df_5 = df2_scores_.copy()

In [ ]:
df_5['Time'] = meta['Time point']

In [ ]:
df_5

In [ ]:
med_df = df_5.groupby(['Time', 'Cardioplegia']).median()

In [ ]:
med_df
std_df = df_5.groupby(['Time', 'Cardioplegia']).std()

In [ ]:
med_df.reset_index(inplace=True)

In [ ]:
med_df

In [ ]:
#sort by "Pre-op", "Op", "Post-op 1", "Post-op 2", "Post-op 3", "Portal vein"
med_df.sort_values(by=['Time'], inplace=True, key=lambda x: x.map({"Pre-op": 0, "Op": 1, "Post-op1": 2, "Post-op2": 3, "Post-op3": 4}))
med_df.sort_values(by=[Group], inplace=True)
med_df.reset_index(drop=True, inplace=True)

In [ ]:
med_df

In [ ]:
#Time trajectory
traject_df = df2_scores_.copy()
traject_df['Time point'] = meta['Time point']



In [ ]:
traject_df

In [ ]:
med_df = traject_df.groupby(['Time point', Group]).median()
err_df = traject_df.groupby(['Time point', Group]).sem()

med_df = med_df.reset_index()
err_df = err_df.reset_index()


In [ ]:
med_df

In [ ]:

med_df.sort_values(by=['Time point'], inplace=True, key=lambda x: x.map({"Pre-op": 0, "Op": 1, "Post-op1": 2, "Post-op2": 3, "Post-op3": 4}))
med_df.sort_values(by=[Group], inplace=True)
med_df.reset_index(drop=True, inplace=True)

err_df.sort_values(by=['Time point'], inplace=True, key=lambda x: x.map({"Pre-op": 0, "Op": 1, "Post-op1": 2, "Post-op2": 3, "Post-op3": 4}))
err_df.sort_values(by=[Group], inplace=True)
err_df.reset_index(drop=True, inplace=True)

In [ ]:
dfx = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/dataset_KOI_Xal.csv')

In [ ]:
dfx

In [ ]:
df_tech = dfx.copy()
idx1 = df_tech[df_tech['AFONOP'] == 1].index
idx2 = df_tech[df_tech['AFONOP'] == 2].index
idx_ = idx1.append(idx2)


In [ ]:
len(idx_)

In [ ]:
df_tech = df_tech.loc[idx_]

In [ ]:
df_tech

In [ ]:

# All

# Import the required python packages including 
# the custom Chemometric Model objects
import numpy as np



from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt

from pyChemometrics.ChemometricsPCA import ChemometricsPCA
from pyChemometrics.ChemometricsScaler import ChemometricsScaler

# Use to obtain same values as in the text
np.random.seed(350)

import os
import plotly.express as px
import plotly.graph_objects as go

from sklearn import decomposition
from sklearn.preprocessing import scale
from pca_ellipse import confidence_ellipse

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from lingress.unipair import Unipair


df = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Code/dataset_1.csv')
#Drop QC samples
df = df.drop(df[df['Technique'] == 'QC'].index)

Group="Technique"
plot_name = "All_1"


#Make directory
# path folder
report_path = '/Volumes/CAS9/Aeiwz/Project/KOI_CC/Report'
project = 'PCA_report_all_1'
# Create directories if they don't exist
os.makedirs('{}/{}'.format(report_path, project), exist_ok=True)
os.makedirs('{}/{}/HTML'.format(report_path, project), exist_ok=True)
os.makedirs('{}/{}/PNG'.format(report_path, project), exist_ok=True)
os.makedirs('{}/{}/Scores'.format(report_path, project), exist_ok=True)
os.makedirs('{}/{}/Loading'.format(report_path, project), exist_ok=True)
os.makedirs('{}/{}/R2'.format(report_path, project), exist_ok=True)
os.makedirs('{}/{}/Trajectory'.format(report_path, project), exist_ok=True)





# path folder
PCA_result_path = '{}/{}'.format(report_path, project)
HTML_save = '{}/HTML'.format(PCA_result_path)
PNG_save = '{}/PNG'.format(PCA_result_path)
Scores_save = '{}/Scores'.format(PCA_result_path)
Loading_save = '{}/Loading'.format(PCA_result_path)
R2_save = '{}/R2'.format(PCA_result_path)
Trajectory_save = '{}/Trajectory'.format(PCA_result_path)

# Import the datasets from the /data directory
# X for the NMR spectra and Y for the 2 outcome variables
test_gr = df

X = test_gr.iloc[:, 113:]
#fill nan with 0
X = X.fillna(0)
meta = test_gr.iloc[:, :113]
Y = test_gr[Group]
Y1 = pd.Categorical(Y).codes
ppm = list(np.ravel(X.columns).astype(float))
# Use pandas Categorical type to generate the dummy enconding of the Y vector (0 and 1) 



import time

from tqdm import tqdm

T1 = time.time()


    # Select the scaling options: 
# Here we are generating 3 scaling objects to explore the effect of scaling in PCA:

# Unit-Variance (UV) scaling:

scale__ = 'Par'
scale_power_ = 0.5

# Mean Centering (MC):
#scaling_object_mc = ChemometricsScaler(scale_power=0)

# Pareto scaling (Par):
# scaling_object_par = ChemometricsScaler(scale_power=0.5)


model_scaler = ChemometricsScaler(scale_power=scale_power_)
model_scaler.fit(X)
model_X = model_scaler.transform(X)

pca_model = decomposition.PCA(n_components=2)
pca_model.fit(model_X)

scores_ = pca_model.transform(model_X)
df_scores_ = pd.DataFrame(scores_, columns=['PC1', 'PC2'])
df_scores_.index = test_gr.index

df2_scores_ = pd.concat([df_scores_, Y], axis=1)

#save PCA score to csv
df2_scores_.to_csv(Scores_save+'/PCA_scores_'+ plot_name +'.csv')

loadings_ = pca_model.components_.T
df_loadings_ = pd.DataFrame(loadings_, columns=['PC1', 'PC2'], index=np.ravel(ppm))
df_loadings_.to_csv(Loading_save + '/Loading_scores ' + plot_name + '.csv')

explained_variance_ = pca_model.explained_variance_ratio_
explained_variance_

explained_variance_ = np.insert(explained_variance_, 0, 0)

cumulative_variance_ = np.cumsum(np.round(explained_variance_, decimals=3))

pc_df_ = pd.DataFrame(['','PC1', 'PC2'], columns=['PC'])
explained_variance_df_ = pd.DataFrame(explained_variance_, columns=['Explained Variance'])
cumulative_variance_df_ = pd.DataFrame(cumulative_variance_, columns=['Cumulative Variance'])

df_explained_variance_ = pd.concat([pc_df_, explained_variance_df_, cumulative_variance_df_], axis=1)
df_explained_variance_.to_csv(R2_save + '/R2 ' + plot_name + '.csv')


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=357)
X_test = model_scaler.transform(X_test)
X_test_pca = pca_model.transform(X_test)

# Inverse transform the test set from the PCA space
X_test_reconstructed = pca_model.inverse_transform(X_test_pca)


# Calculate Q2 score for the test set
q2_test = r2_score(X_test, X_test_reconstructed)
        

# Plot

# https://plotly.com/python/bar-charts/

fig = px.bar(df_explained_variance_, 
            x='PC', y='Explained Variance',
            text='Explained Variance',
            width=800, height=600,
            title='Explained Variance ({} scaling)'.format(scale__))
fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font=dict(size=15))
fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
#fig.show()
fig.write_image(PNG_save + "/Explained Variance " + plot_name + ".png")
fig.write_html(HTML_save + "/Explained Variance " + plot_name + ".html")

# https://plotly.com/python/creating-and-updating-figures/
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_explained_variance_['PC'],
        y=df_explained_variance_['Cumulative Variance'],
        marker=dict(size=15, color="LightSeaGreen"),
        name='R<sup>2</sup>X (Cum)'
    ))

fig.add_trace(
    go.Bar(
        x=df_explained_variance_['PC'],
        y=df_explained_variance_['Explained Variance'],
        marker=dict(color="RoyalBlue"),
        name='R<sup>2</sup>X',
        text=np.round(df_explained_variance_['Explained Variance'], decimals=3)
    ))
fig.update_layout(width=800, height=600,
                title='Explained Variance and Cumulative Variance ' + plot_name)
fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

#fig.show()
fig.write_image(PNG_save + "/Explained Variance + Cumulative Variance " + plot_name + ".png")
fig.write_html(HTML_save + "/Explained Variance + Cumulative Variance " + plot_name + ".html")


colour_dict = {
                "Off pump CABG (OPCAB)": "#E91E63",        
                "On pump CABG (ONCAB)": "#FF9800",
                "custodiol": "#FFEB3B",       
                "del Nido": "#9C27B0",
                "st. thomas": "#03A9F4",
                "No med": "#4CAF50",        
                "7": "#B30000",
                "8": "#3F51B5"
                }





# PCA plot
pca_label = df2_scores_.index


fig = px.scatter(df2_scores_, x='PC1', y='PC2', symbol=meta['Time point'],
                    
                    symbol_map={'Pre-op': 'circle',
                        'Op': 'diamond',
                        'Post-op1': 'square',
                        'Post-op2': 'triangle-up',
                        'Post-op3': 'triangle-down'},
                    
                color=Group,
                color_discrete_map=colour_dict, 
                title='<b>PCA Scores Plot ({} Scaling)<b>'.format(scale__), 
                height=900, width=1300,
                labels={"PC1": "PC1 R<sup>2</sup>X: {} %".format(np.round(df_explained_variance_.iloc[1,1]*100, decimals=2)),
                        "PC2": "PC2 R<sup>2</sup>X: {} %".format(np.round(df_explained_variance_.iloc[2,1]*100, decimals=2))})

#fig.add_annotation(yref = 'paper', y = -1.06, xref = 'paper', x=1.06 , text='Q2' +' = {}'.format(np.round(df_explained_variance_.iloc[2,2], decimals=2)))
#fig.update_annotations(font = {
#    'size': 20}, showarrow=False)

#set data point fill alpha with boarder in each color
fig.update_traces(marker=dict(size=35, opacity=0.7, line=dict(width=2, color='DarkSlateGrey')))

fig.add_annotation(dict(font=dict(color="black",size=20),
                        #x=x_loc,
                        x=1.0,
                        y=0.05,
                        showarrow=False,
                        text='<b>R<sup>2</sup>X (Cum): {}%<b>'.format(np.round(df_explained_variance_.iloc[2,2]*100, decimals=2)),
                        textangle=0,
                        xref="paper",
                        yref="paper"),
                        # set alignment of text to left side of entry
                        align="left")

fig.add_annotation(dict(font=dict(color="black",size=20),
                        #x=x_loc,
                        x=1.0,
                        y=0.01,
                        showarrow=False,
                        text='<b>Q<sup>2</sup>X (Cum): {}%<b>'.format(np.round(q2_test*100, decimals=2)),
                        textangle=0,
                        xref="paper",
                        yref="paper"),
                        # set alignment of text to left side of entry
                        align="left")



fig.update_traces(marker=dict(size=20))
#fig.update_traces(textposition='top center') #Text label position

#fig.update_traces(marker=dict(size=12, color=Y1_color, marker=Y2_marker))
fig.add_shape(type='path',
            path=confidence_ellipse(df2_scores_['PC1'], df2_scores_['PC2']))



fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    title={
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font=dict(size=20))
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')

#fig.show()
fig.write_image(PNG_save + "/PCA " + plot_name + ".png")
fig.write_html(HTML_save + "/PCA " + plot_name + ".html")


# Loading plot
loadings_label = df_loadings_.index


fig = px.line(df_loadings_, x=loadings_label, y=['PC1', 'PC2'],
                height=600, width=1800,
                title='Loadings ' + plot_name
                )

fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
fig.update_layout(title={'y':0.95,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                font=dict(size=20))

fig.update_layout(scene={'xaxis': {'autorange': 'reversed'}})
        
fig.update_traces(marker=dict(size=1))
fig.update_layout(xaxis_title="𝛿<sub>H</sub> in ppm")
#fig.show()

fig.write_image(PNG_save + "/Loading " + plot_name + ".png")
fig.write_html(HTML_save + "/Loading " + plot_name + ".html")


#Time trajectory
traject_df = df2_scores_.copy()
traject_df['Time point'] = meta['Time point']
med_df = traject_df.groupby(['Time point', Group]).median()
err_df = traject_df.groupby(['Time point', Group]).sem()

med_df = med_df.reset_index()
err_df = err_df.reset_index()


med_df.sort_values(by=['Time point'], inplace=True, key=lambda x: x.map({"Pre-op": 0, "Op": 1, "Post-op1": 2, "Post-op2": 3, "Post-op3": 4}))
med_df.sort_values(by=[Group], inplace=True)
med_df.reset_index(drop=True, inplace=True)

err_df.sort_values(by=['Time point'], inplace=True, key=lambda x: x.map({"Pre-op": 0, "Op": 1, "Post-op1": 2, "Post-op2": 3, "Post-op3": 4}))
err_df.sort_values(by=[Group], inplace=True)
err_df.reset_index(drop=True, inplace=True)


list_label = med_df[Group].unique()
#line 1
if list_label[0] == 'Off pump CABG (OPCAB)': 
    colour1 = '#E91E63'
elif list_label[0] == 'On pump CABG (ONCAB)':
    colour1 = '#FF9800'
elif list_label[0] == 'custodiol':
    colour1 = '#FFEB3B'
elif list_label[0] == 'del Nido':
    colour1 = '#9C27B0'
elif list_label[0] == 'st. thomas':
    colour1 = '#03A9F4'
elif list_label[0] == 'No med':
    colour1 = '#4CAF50'

#line 2
if list_label[1] == 'Off pump CABG (OPCAB)':
    colour2 = '#E91E63'
elif list_label[1] == 'On pump CABG (ONCAB)':
    colour2 = '#FF9800'
elif list_label[1] == 'custodiol':
    colour2 = '#FFEB3B'
elif list_label[1] == 'del Nido':
    colour2 = '#9C27B0'
elif list_label[1] == 'st. thomas':
    colour2 = '#03A9F4'
elif list_label[1] == 'No med':
    colour2 = '#4CAF50'
    



fig = px.line(med_df, x='PC1', y='PC2', line_group='Time point', error_x=err_df["PC1"], error_y=err_df["PC2"],
                color=Group, 
                symbol='Time point',
                color_discrete_map=colour_dict,
                symbol_map={'Pre-op': 'circle',
                        'Op': 'diamond',
                        'Post-op1': 'square',
                        'Post-op2': 'triangle-up',
                        'Post-op3': 'triangle-down'}, 
                title='<b>Principle component analysis (Pareto scaling)<b>', 
                height=900, width=1300,
                labels={"PC1": "PC1 R<sup>2</sup>X: {} %".format(24.3),
                        "PC2": "PC2 R<sup>2</sup>X: {} %".format(15.0)})


# create a new trace for the connecting line
fig.add_trace(go.Scatter(
    x=med_df.loc[0:4, "PC1"], # x-coordinates of the line
    y=med_df.loc[0:4, "PC2"], # y-coordinates of the line
    mode='lines', # specify the trace type as lines
    line=dict(color=colour1, width=2), # set the color and width of the line
    showlegend=False # hide the trace from the legend
))


# create a new trace for the connecting line
fig.add_trace(go.Scatter(
    x=med_df.loc[5:9, "PC1"], # x-coordinates of the line
    y=med_df.loc[5:9, "PC2"], # y-coordinates of the line
    mode='lines', # specify the trace type as lines
    line=dict(color=colour2, width=2), # set the color and width of the line
    showlegend=False # hide the trace from the legend
))
'''
# create a new trace for the connecting line
fig.add_trace(go.Scatter(
    x=df_score_mean.iloc[4:8, 0], # x-coordinates of the line
    y=df_score_mean.iloc[4:8, 2], # y-coordinates of the line
    mode='lines', # specify the trace type as lines
    line=dict(color='#84CC56', width=2), # set the color and width of the line
    showlegend=False # hide the trace from the legend
))


# create a new trace for the connecting line
fig.add_trace(go.Scatter(
    x=df_score_mean.iloc[8:12, 0], # x-coordinates of the line
    y=df_score_mean.iloc[8:12, 2], # y-coordinates of the line
    mode='lines', # specify the trace type as lines
    line=dict(color='#CA83CC', width=2), # set the color and width of the line
    showlegend=False # hide the trace from the legend
))

# create a new trace for the connecting line
fig.add_trace(go.Scatter(
    x=df_score_mean.iloc[8:12, 0], # x-coordinates of the line
    y=df_score_mean.iloc[8:12, 2], # y-coordinates of the line
    mode='lines', # specify the trace type as lines
    line=dict(color='#6AE022', width=2), # set the color and width of the line
    showlegend=False # hide the trace from the legend
))

'''





fig.add_annotation(dict(font=dict(color="black",size=20),
                        #x=x_loc,
                        x=1.0,
                        y=0.05,
                        showarrow=False,
                        text='<b>R<sup>2</sup>X (Cum): {}%<b>'.format(np.round(df_explained_variance_.iloc[2,2]*100, decimals=2)),
                        textangle=0,
                        xref="paper",
                        yref="paper"),
                        # set alignment of text to left side of entry
                        align="left")

fig.add_annotation(dict(font=dict(color="black",size=20),
                        #x=x_loc,
                        x=1.0,
                        y=0.01,
                        showarrow=False,
                        text='<b>Q<sup>2</sup>X (Cum): {}%<b>'.format(np.round(q2_test*100, decimals=2)),
                        textangle=0,
                        xref="paper",
                        yref="paper"),
                        # set alignment of text to left side of entry
                        align="left")

fig.update_traces(marker=dict(size=20))
#fig.update_traces(textposition='top center') #Text label position

#fig.update_traces(marker=dict(size=12, color=Y1_color, marker=Y2_marker))
fig.add_shape(type='path',
            path=confidence_ellipse(med_df['PC1'],med_df['PC2']))



#update axis as scitifics
fig.update_xaxes(tickformat=".1e")
fig.update_yaxes(tickformat=".1e")



fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font=dict(size=20))
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
#fig.show()

fig.write_image("{}/Save_PCA_trajectory_".format(Trajectory_save) + plot_name + ".png")
fig.write_html("{}/Save_PCA_trajectory_".format(Trajectory_save) + plot_name + ".html")




T2 = time.time()

print('{} Done /n Time taken: {} seconds'.format(plot_name, T2-T1))




In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dfx = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/KOI_Xal.csv')
ppm = pd.read_csv('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/KOI_ppm.csv')
meta = pd.read_excel('/Volumes/CAS9/Aeiwz/Project/KOI_CC/Data/RunOrder_CKVremodel.xlsx', sheet_name='Data (Original)')

In [ ]:
meta.iloc[:, :15]

In [ ]:
df = dfx.copy()
ppm = list(np.ravel(ppm))
df.columns = ppm
dataset = pd.concat([meta, df], axis=1)

In [ ]:
dataset

In [ ]:
columns_list = meta.columns

In [ ]:
columns_list[15]

In [ ]:
dataset =dataset.drop(dataset[dataset['Technique'] == 99].index)

In [ ]:
dataset['Technique'].unique()

In [ ]:
project = dataset['Cardioplegia'].unique()

In [ ]:
dataset.iloc[:, 10:20]


In [ ]:
for f in project:
    data= dataset[dataset['Cardioplegia'] == f]
        
        
        


    # Import necessary libraries
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.metrics import accuracy_score, roc_auc_score
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    from sklearn.decomposition import PCA
    from sklearn.cross_decomposition import PLSRegression
    from sklearn.model_selection import permutation_test_score
    import matplotlib.pyplot as plt
    from sklearn.metrics import r2_score
    from sklearn.utils import shuffle
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.offline as pyo
    import cross_validation
    import plotting
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error

    import warnings
    warnings.filterwarnings('ignore')

    from pyChemometrics import ChemometricsScaler

    import os
    from lingress.unipair import Unipair




    #Make directory
    # path folder
    path_ = '/Volumes/CAS9/Aeiwz/Project/KOI_CC/Report/{}'.format(f)

    os.makedirs(path_, exist_ok=True)

    # Create directories if they don't exist
    os.makedirs('{}/element'.format(path_), exist_ok=True)
    os.makedirs('{}/element/hist_plot'.format(path_), exist_ok=True)
    os.makedirs('{}/element/loading_plot'.format(path_), exist_ok=True)
    os.makedirs('{}/element/score_plot'.format(path_), exist_ok=True)
    os.makedirs('{}/element/s_plot'.format(path_), exist_ok=True)
    os.makedirs('{}/main'.format(path_), exist_ok=True)
    os.makedirs('{}/element/Lingress'.format(path_), exist_ok=True)

    # Import the datasets from the /data directory


    # X for the NMR spectra and Y for the 2 outcome variables
    path_save = "{}/element".format(path_)
    #test group
    df = data
    #Drop QC samples


    Group="Time point"

    test_index = Unipair.indexing(meta = df, column_name=Group)



    c_list = test_index




    for i in range(len(c_list)):
        


        test_select = df.copy()
        test_select = test_select.loc[c_list[i]]
        
        
        X = test_select.iloc[:, 119:]
        df_X = X
        #fill nan with 0
        X = X.fillna(0)
        meta = test_select.iloc[:, :119]
        Y = test_select[Group]
        Y1 = pd.Categorical(Y).codes
        ppm = list(np.ravel(X.columns).astype(float))
        # Use pandas Categorical type to generate the dummy enconding of the Y vector (0 and 1) 
        name = str(i)
    

        spectra = test_select.iloc[:, 119:]
        ppm = list(spectra.columns.astype(float))
        X = spectra.values
        y = Y
        
        # Create a pipeline with data preprocessing and OPLS-DA model
        pipeline = Pipeline([
                                ('scale', ChemometricsScaler(scale_power=0.5)),
                                ('oplsda', PLSRegression(n_components=2)),
                                ('opls', cross_validation.CrossValidation(kfold=3, estimator='opls', scaler='pareto'))
                            ])

        oplsda = pipeline.named_steps['oplsda']
        cv = pipeline.named_steps['opls']
        cv.fit(X, y)

        oplsda.fit(X, pd.Categorical(y).codes)
        n_permutate = 1000

        # Permutation test to assess the significance of the model
        acc_score, permutation_scores, p_value = permutation_test_score(
        pipeline.named_steps['oplsda'], X, pd.Categorical(y).codes, cv=3, n_permutations=n_permutate, n_jobs=-1, random_state=57, verbose=10)


        s_scores_df = pd.DataFrame({'correlation': cv.correlation,'covariance': cv.covariance}, index=ppm)
        df_opls_scores = pd.DataFrame({'t_scores': cv.scores, 't_ortho': cv.orthogonal_score, 't_pred': cv.predictive_score, 'label': y})

            
        colour_dict = {
                "Off pump CABG (OPCAB)": "#E91E63",        
                "On pump CABG (ONCAB)": "#FF9800",
                "custodiol": "#FFEB3B",       
                "del Nido": "#9C27B0",
                "st. thomas": "#03A9F4",
                "No med": "#4CAF50",        
                "7": "#B30000",
                "8": "#3F51B5"
                }

        #Visualise
        from pca_ellipse import confidence_ellipse
        fig = px.scatter(df_opls_scores, x='t_scores', y='t_ortho', 
                        color='label', 
                        color_discrete_map=colour_dict, 
                        title='<b>OPLS-DA Scores Plot<b>', 
                        height=900, width=1300,
                        labels={
                            't_pred': 't<sub>predict</sub>',
                            't_ortho': 't<sub>orthogonal</sub>',
                            't_scores': 't<sub>scores</sub>',
                            'label': 'Intervention'}
                        )

        #fig.add_annotation(yref = 'paper', y = -1.06, xref = 'paper', x=1.06 , text='Q2' +' = {}'.format(np.round(df_explained_variance_.iloc[2,2], decimals=2)))
        #fig.update_annotations(font = {
        #    'size': 20}, showarrow=False)

        #set data point fill alpha with boarder in each color
        fig.update_traces(marker=dict(size=35, opacity=0.7, line=dict(width=2, color='DarkSlateGrey')))

        fig.add_annotation(dict(font=dict(color="black",size=20),
                                #x=x_loc,
                                x=0,
                                y=1.04+0.05,
                                showarrow=False,
                                text='<b>R<sup>2</sup>X: {}%<b>'.format(np.round(cv.R2Xcorr*100, decimals=2)),
                                textangle=0,
                                xref="paper",
                                yref="paper"),
                                # set alignment of text to left side of entry
                                align="left")

        fig.add_annotation(dict(font=dict(color="black",size=20),
                                #x=x_loc,
                                x=0,
                                y=1.0+0.05,
                                showarrow=False,
                                text='<b>R<sup>2</sup>Y: {}%<b>'.format(np.round(cv.R2y*100, decimals=2)),
                                textangle=0,
                                xref="paper",
                                yref="paper"),
                                # set alignment of text to left side of entry
                                align="left")
        fig.add_annotation(dict(font=dict(color="black",size=20),
                                #x=x_loc,
                                x=0,
                                y=1.08+0.05,
                                showarrow=False,
                                text='<b>Q<sup>2</sup>: {}%<b>'.format(np.round(cv.q2*100, decimals=2)),
                                textangle=0,
                                xref="paper",
                                yref="paper"),
                                # set alignment of text to left side of entry
                                align="left")

        fig.add_shape(type='path',
                path=confidence_ellipse(df_opls_scores['t_scores'], df_opls_scores['t_ortho']))


        fig.update_traces(marker=dict(size=35))
        #fig.update_traces(textposition='top center') #Text label position
        #change M to 10^6
        fig.update_yaxes(tickformat=",.0")
        fig.update_xaxes(tickformat=",.0")

        #fig.update_traces(marker=dict(size=12, color=Y1_color, marker=Y2_marker))

        fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
        fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
        fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
        fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
        fig.update_layout(
            title={
                'y':1,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            font=dict(size=20))
        fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')

        #fig.show()
        fig.write_image("{}/score_plot/score_plot_{}.png".format(path_save, name))
        fig.write_html("{}/score_plot/score_plot{}.html".format(path_save, name))




        #Histrogram
        #Plot histogram of permutation scores
        fig = px.histogram(permutation_scores, nbins=50, height=500, width=1000, 
                        title='<b>Permutation scores<b>',
                        labels={'value': 'Accuracy score', 
                                'count': 'Frequency'})
        #add dashed line to indicate the accuracy score of the model line y location is maximum count of histogram
        fig.add_shape(type='line', yref='paper', y0=0, y1=1, xref='x', x0=acc_score, x1=acc_score, line=dict(dash='dash', color='red', width=3))


        fig.add_annotation(dict(font=dict(color="black",size=14),
                                #x=x_loc,
                                x=0,
                                y=1.25,
                                #y=1.18,
                                showarrow=False,
                                text='Number of permutation: {}'.format(n_permutate),
                                textangle=0,
                                xref="paper",
                                yref="paper"),
                                # set alignment of text to left side of entry
                                align="left")

        fig.add_annotation(dict(font=dict(color="black",size=14),
                                #x=x_loc,
                                x=0,
                                y=1.18,
                                showarrow=False,
                                text='Accuracy score: {}'.format(np.round(acc_score, decimals=3)),
                                textangle=0,
                                xref="paper",
                                yref="paper"),
                                # set alignment of text to left side of entry
                                align="left")
        fig.add_annotation(dict(font=dict(color="black",size=14),
                                #x=x_loc,
                                x=0,
                                y=1.11,
                                showarrow=False,
                                text='<i>p-value</i>: {}'.format(np.round(p_value, decimals=6)),
                                textangle=0,
                                xref="paper",
                                yref="paper"),
                                # set alignment of text to left side of entry
                                align="left")

        fig.update_layout(showlegend=False)

        fig.update_layout(title_x=0.5)

        #fig.show()
        fig.write_image("{}/hist_plot/Permutation_scores_{}.png".format(path_save, name))
        fig.write_html("{}/hist_plot/Permutation_scores_{}.html".format(path_save, name))
        
        
        
        #S plot
        # sub-plot covariance for x and correlation for y S-plot using plotly, color by covariance with jet colormap
        #setup figure size


        fig = px.scatter(s_scores_df, x='covariance', y='correlation', color='covariance', range_color=[-1,1],
                        color_continuous_scale='jet', text=s_scores_df.index, height=900, width=2000)
        fig.update_layout(title='<b>S-plot</b>', xaxis_title='Covariance', yaxis_title='Correlation')

        #add line of axis and set color to black and line width to 2 pixel
        fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
        fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
        #Add tick width to 2 pixel
        fig.update_xaxes(tickwidth=2)
        fig.update_yaxes(tickwidth=2)
        fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
        fig.update_yaxes(tickformat=",.0")
        #fig.update_xaxes(tickformat=",.0")
        fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
        fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
        fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
        fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
        fig.update_layout(
            title={
                'y':1,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            font=dict(size=20))
        #Set font size to 20
        #Set marker size to 5 pixel
        fig.update_traces(marker=dict(size=14))
        #fig.show()
        fig.write_image("{}/s_plot/S_plot_{}.png".format(path_save, name))
        fig.write_html("{}/s_plot/S_plot_{}.html".format(path_save, name))
        

        #Loadings plot
        
        # X * 1 when correlation is positive, X * -1 when correlation is negative
        def median_corr(X):
            X_corr = np.median(X, axis=0)
            X_corr = X_corr * np.sign(s_scores_df['correlation'])
            return X_corr

        X2 = median_corr(X)

        fig = px.scatter(s_scores_df, x=ppm, y=X2, color='covariance', color_continuous_scale='jet', text=s_scores_df.index, height=500, width=2000)

        fig.update_traces(marker=dict(size=3))
        fig.update_xaxes(autorange="reversed")
        fig.update_layout(title='<b>Median spectra</b>', xaxis_title='ppm', yaxis_title='Correlation')
        fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
        fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
        #Add tick width to 2 pixel
        fig.update_xaxes(tickwidth=2)
        fig.update_yaxes(tickwidth=2)

        fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
        fig.update_yaxes(tickformat=",.0")
        #fig.update_xaxes(tickformat=",.0")
        fig.update_layout(
            title={
                'y':1,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            font=dict(size=20))
        #Set marker size to 5 pixel
        fig.update_traces(marker=dict(size=3))
        #fig.show()
        fig.write_image("{}/loading_plot/loadings_plot_{}.png".format(path_save, name))
        fig.write_html("{}/loading_plot/loadings_plot_{}.html".format(path_save, name))
        
        from lingress.Lingress import lin_regression
        lin_mod = lin_regression(x=df_X, target=meta[Group], label=meta[Group], features_name=ppm)
        lin_mod.create_dataset()
        lin_mod.fit_model(adj_method='fdr_bh')
        report = lin_mod.report()
        report.to_csv("{}/Lingress/lingress_report_{}.csv".format(path_save, name))
        lin_mod.volcano_plot()
        lin_mod.png_plot(plot_name="lingress_volcano_plot_{}_{}".format(f,name), path_save=path_save)
        lin_mod.html_plot(plot_name="lingress_volcano_plot_{}_{}".format(f,name), path_save=path_save)



# **Generate HTML file**

In [10]:
# _*_ coding: utf-8 _*_


import sys
import os
import re
import shutil
from glob import glob
import pandas as pd

__author__ = "aeiwz"


class gen_page:

    def __init__(self, data_path):
        '''
        This function takes in the path to the data folder and returns the HTML files for the OPLS-DA plots.
        Parameters
        ----------
        data_path: str
            The path to the data folder.
        gen_page(data_path).get_files()
        '''
        self.data_path = data_path

        if data_path[-1] == '/':
            #remove the last /
            data_path = data_path[:-1]
            
        else:
            data_path = data_path

        

        #check data_path is a string
        if not isinstance(data_path, str):
            raise ValueError("data_path should be a string")
        
        #check data_path is a directory
        if not os.path.isdir(data_path):
            raise ValueError("data_path should be a directory")

        #check if data_path is empty
        if not os.listdir(data_path):
            raise ValueError("data_path should not be empty")

        #check if data_path contains the necessary files
        if not os.path.exists(data_path+'/element/hist_plot'):
            raise ValueError("data_path should contain a folder named 'element' with a folder named 'hist_plot'")
        if not os.path.exists(data_path+'/element/Lingress'):
            raise ValueError("data_path should contain a folder named 'element' with a folder named 'Lingress'")
        if not os.path.exists(data_path+'/element/loading_plot'):
            raise ValueError("data_path should contain a folder named 'element' with a folder named 'loading_plot'")
        if not os.path.exists(data_path+'/element/s_plot'):
            raise ValueError("data_path should contain a folder named 'element' with a folder named 's_plot'")
        if not os.path.exists(data_path+'/element/score_plot'):
            raise ValueError("data_path should contain a folder named 'element' with a folder named 'score_plot'")



        #change directory to data_path
        os.chdir(data_path)

    def get_files(self):
        
        data_path = self.data_path


        hist_plot = glob(pathname= data_path+'/element/hist_plot/*.html')
        Lingress_ = glob(pathname= data_path+'/element/Lingress/*.html')
        loading_plot = glob(pathname= data_path+'/element/loading_plot/*.html')
        s_plot = glob(pathname= data_path+'/element/s_plot/*.html')
        score_plot = glob(pathname= data_path+'/element/score_plot/*.html')

        files = pd.DataFrame({'hist_plot': hist_plot, 'Lingress': Lingress_, 'loading_plot': loading_plot, 's_plot': s_plot, 'score_plot': score_plot})

        # Get the name of the files
        files['names'] = files['hist_plot'].str.split('/').str[-1].str.split('.').str[0]
        files['names'] = files['names'].str.replace('Permutation_scores_','')
        files['names'] = files['names'].str.replace(' ','_')
        #replace value in dataframe with ..
        files = files.replace(to_replace=data_path, value='..', regex=True)

        html_content_list = []

        # Iterate through the name and file_ lists to create HTML files
        for i in range(len(files)):
            html_content = f"""

        <!DOCTYPE html>
        <html lang="en">
        <head>
        <title>OPLS-DA</title>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="description" content="HTML5 website template">
        <meta name="keywords" content="CASIX, template, html, sass, jquery">
        <meta name="author" content="CASIX">

        <link rel="ix-icon" type="image/png" href="assets/img/logo.PNG">
        <link rel="shortcut icon" type="image/png" href="assets/img/logo.PNG">
        </head>
        <body>

        <body>

            <div class="container">
                <iframe src="{files['score_plot'][i]}" 
                frameborder="0" 
                width="100%" 
                height="1000"></iframe>
            </div>
            
            <div class="container">
                <iframe src="{files['loading_plot'][i]}" 
                frameborder="0" 
                width="100%" 
                height="1000"></iframe>
            </div>
            
            <div class="container">
                <iframe src="{files['s_plot'][i]}" 
                frameborder="0" 
                width="100%" 
                height="600"></iframe>
            </div>

            <div class="container">
                <iframe src="{files['hist_plot'][i]}" 
                frameborder="0" 
                width="100%" height="600"></iframe>

            </div>

                <div class="container">
                <iframe src="{files['Lingress'][i]}" 
                frameborder="0" 
                width="100%" height="600"></iframe>
            </div>


        </body>
        </html>

            
            """
            html_content_list.append(html_content)
        
        for i in range(len(files)):
            # Write the HTML content to the file
            file_path = f"./main/oplsda_{files['names'][i]}.html"
            with open(file_path, "w") as html_file:
                html_file.write(html_content_list[i])

        return print('HTML files created')

            


In [17]:
file_path = '/Volumes/CAS9/Aeiwz/Documents/Thesis/Analyse/Report/OPLSDA_U'

In [18]:
gen_page(data_path=file_path).get_files()

AttributeError: Can only use .str accessor with string values!

In [20]:

data = pd.read_csv('/Volumes/CAS9/Aeiwz/Documents/Thesis/Analyse/Dataset/U_noesy_pqn.csv')


In [26]:
Class_ = ['sham ad libitum, 2-wk post-op', 'CR + INT777 (H), 2-wk post-op', 'CR + INT777 (L), 2-wk post-op', 'sham ad libitum, 1-wk post-op', 'CR + INT777 (H), 1-wk post-op', 'CR + INT777 (L), 2-wk post-op', 'CR + INT777 (H), 4-wk post-op', 'CR + INT777 (L), 1-wk post-op', 'CR + INT777 (L), 1-wk pre-op', 'CR + INT777 (L), 4-wk post-op', 'sham ad libitum, 1-wk post-op', 'sham ad libitum, 1-wk pre-op', 'CR + INT777 (H), 4-wk post-op', 'sham ad libitum, 1-wk pre-op', 'CR + INT777 (L), 4-wk post-op', 'sham ad libitum, 4-wk post-op', 'CR + INT777 (L), 1-wk pre-op', 'CR + INT777 (H), 1-wk pre-op', 'CR + INT777 (H), 4-wk post-op', 'sham ad libitum, 1-wk pre-op', 'CR + INT777 (H), 1-wk pre-op', 'sham ad libitum, 2-wk post-op', 'CR + INT777 (H), 4-wk post-op', 'CR + INT777 (L), 4-wk post-op', 'sham ad libitum, 4-wk post-op', 'CR + INT777 (H), 2-wk post-op', 'CR + INT777 (H), 4-wk post-op', 'CR + INT777 (L), 4-wk post-op', 'CR + INT777 (L), 4-wk post-op', 'sham ad libitum, 2-wk post-op', 'CR + INT777 (H), 2-wk post-op', 'sham ad libitum, 2-wk post-op', 'sham ad libitum, 1-wk post-op', 'CR + INT777 (H), 2-wk post-op', 'CR + INT777 (L), 1-wk post-op', 'CR + INT777 (L), 1-wk pre-op', 'sham ad libitum, 1-wk pre-op', 'CR + INT777 (H), 1-wk post-op', 'sham ad libitum, 1-wk pre-op', 'sham ad libitum, 2-wk post-op', 'CR + INT777 (L), 1-wk post-op', 'sham ad libitum, 4-wk post-op', 'CR + INT777 (L), 2-wk post-op', 'CR + INT777 (H), 1-wk post-op', 'CR + INT777 (H), 1-wk pre-op', 'sham ad libitum, 1-wk post-op', 'sham ad libitum, 4-wk post-op', 'sham ad libitum, 4-wk post-op', 'CR + INT777 (L), 2-wk post-op', 'CR + INT777 (L), 1-wk pre-op', 'CR + INT777 (L), 1-wk post-op', 'CR + INT777 (H), 1-wk pre-op', 'CR + INT777 (L), 1-wk pre-op', 'sham ad libitum, 1-wk post-op', 'CR + INT777 (L), 2-wk post-op', 'CR + INT777 (L), 1-wk post-op', 'CR + INT777 (H), 2-wk post-op', 'CR + INT777 (H), 1-wk pre-op', 'CR + INT777 (H), 1-wk post-op', 'CR + INT777 (H), 1-wk post-op']

In [27]:
data['Class'] = Class_

In [30]:

# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import permutation_test_score
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.utils import shuffle
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import cross_validation
import plotting
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

from pyChemometrics import ChemometricsScaler

import os
from lingress.unipair import Unipair




#Make directory
# path folder

data = pd.read_csv('/Volumes/CAS9/Aeiwz/Documents/Thesis/Analyse/Dataset/U_noesy_pqn.csv')

#exclude INT_H in index number 59
data = data.drop([59])


pair = Unipair(meta=data, column_name='Class')

dataset = pair.get_dataset()

names = pair.get_name()


path_ = "/Volumes/CAS9/Aeiwz/Documents/Thesis/Analyse/Report/OPLSDA_U"
os.makedirs(path_, exist_ok=True)

# Create directories if they don't exist
os.makedirs('{}/element'.format(path_), exist_ok=True)
os.makedirs('{}/element/hist_plot'.format(path_), exist_ok=True)
os.makedirs('{}/element/loading_plot'.format(path_), exist_ok=True)
os.makedirs('{}/element/score_plot'.format(path_), exist_ok=True)
os.makedirs('{}/element/s_plot'.format(path_), exist_ok=True)
os.makedirs('{}/main'.format(path_), exist_ok=True)
os.makedirs('{}/element/Lingress'.format(path_), exist_ok=True)

# Import the datasets from the /data directory


# X for the NMR spectra and Y for the 2 outcome variables
path_save = "{}/element".format(path_)
#test group
df = data
#Drop QC samples
#df = df.drop(df[df['Technique'] == 'QC'].index)

Group="Class"



for i in range(len(dataset)):
    

    plot_name = names[i]


    test_select = dataset[i]

    
    
    X = test_select.iloc[:, 14:]
    df_X = X
    #fill nan with 0
    X = X.fillna(0)
    meta = test_select.iloc[:, :14]
    Y = test_select[Group]
    Y1 = pd.Categorical(Y).codes
    ppm = list(np.ravel(X.columns).astype(float))
    # Use pandas Categorical type to generate the dummy enconding of the Y vector (0 and 1) 
    name = names[i]
   

    spectra = test_select.iloc[:, 14:]
    ppm = list(spectra.columns.astype(float))
    X = spectra.values
    y = Y
    
    # Create a pipeline with data preprocessing and OPLS-DA model
    pipeline = Pipeline([
                            ('scale', ChemometricsScaler(scale_power=0.5)),
                            ('oplsda', PLSRegression(n_components=2)),
                            ('opls', cross_validation.CrossValidation(kfold=3, estimator='opls', scaler='pareto'))
                         ])

    oplsda = pipeline.named_steps['oplsda']
    cv = pipeline.named_steps['opls']
    cv.fit(X, y)

    oplsda.fit(X, pd.Categorical(y).codes)
    n_permutate = 1000

    # Permutation test to assess the significance of the model
    acc_score, permutation_scores, p_value = permutation_test_score(
    pipeline.named_steps['oplsda'], X, pd.Categorical(y).codes, cv=3, n_permutations=n_permutate, n_jobs=-1, random_state=57, verbose=10)


    s_scores_df = pd.DataFrame({'correlation': cv.correlation,'covariance': cv.covariance}, index=ppm)
    df_opls_scores = pd.DataFrame({'t_scores': cv.scores, 't_ortho': cv.orthogonal_score, 't_pred': cv.predictive_score, 'label': y})

        
    colour_dict = {
                    "sham ad libitum": "#F55D4D",        
                    "CR + INT777 (H)": "#58E6BE",
                    "CR + INT777 (L)": "#4E8BF5",       
                    }
    
    symbol_dict = {'1-wk pre-op': 'circle',
                    '1-wk post-op': 'diamond',
                    '2-wk post-op': 'square',
                    '4-wk post-op': 'triangle-up'
                    }


    #Visualise
    from pca_ellipse import confidence_ellipse
    fig = px.scatter(df_opls_scores, x='t_scores', y='t_ortho', symbol=meta['Time point'], 
                
                    symbol_map=symbol_dict,
                
                    color='label', 
                    color_discrete_map=colour_dict, 
                    title='<b>OPLS-DA Scores Plot<b>', 
                    height=900, width=1300,
                    labels={
                        't_pred': 't<sub>predict</sub>',
                        't_ortho': 't<sub>orthogonal</sub>',
                        't_scores': 't<sub>scores</sub>',
                        'label': 'Intervention'}
                    )

    #fig.add_annotation(yref = 'paper', y = -1.06, xref = 'paper', x=1.06 , text='Q2' +' = {}'.format(np.round(df_explained_variance_.iloc[2,2], decimals=2)))
    #fig.update_annotations(font = {
    #    'size': 20}, showarrow=False)

    #set data point fill alpha with boarder in each color
    fig.update_traces(marker=dict(size=35, opacity=0.7, line=dict(width=2, color='DarkSlateGrey')))

    fig.add_annotation(dict(font=dict(color="black",size=20),
                            #x=x_loc,
                            x=0,
                            y=1.04+0.05,
                            showarrow=False,
                            text='<b>R<sup>2</sup>X: {}%<b>'.format(np.round(cv.R2Xcorr*100, decimals=2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")

    fig.add_annotation(dict(font=dict(color="black",size=20),
                            #x=x_loc,
                            x=0,
                            y=1.0+0.05,
                            showarrow=False,
                            text='<b>R<sup>2</sup>Y: {}%<b>'.format(np.round(cv.R2y*100, decimals=2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")
    fig.add_annotation(dict(font=dict(color="black",size=20),
                            #x=x_loc,
                            x=0,
                            y=1.08+0.05,
                            showarrow=False,
                            text='<b>Q<sup>2</sup>: {}%<b>'.format(np.round(cv.q2*100, decimals=2)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")

    fig.add_shape(type='path',
            path=confidence_ellipse(df_opls_scores['t_scores'], df_opls_scores['t_ortho']))


    fig.update_traces(marker=dict(size=35))
    #fig.update_traces(textposition='top center') #Text label position
    #change M to 10^6
    fig.update_yaxes(tickformat=",.0")
    fig.update_xaxes(tickformat=",.0")

    #fig.update_traces(marker=dict(size=12, color=Y1_color, marker=Y2_marker))

    fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_layout(
        title={
            'y':1,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        font=dict(size=20))
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')

    #fig.show()
    fig.write_image("{}/score_plot/score_plot_{}.png".format(path_save, name))
    fig.write_html("{}/score_plot/score_plot{}.html".format(path_save, name))




    #Histrogram
    #Plot histogram of permutation scores
    fig = px.histogram(permutation_scores, nbins=50, height=500, width=1000, 
                    title='<b>Permutation scores<b>',
                    labels={'value': 'Accuracy score', 
                            'count': 'Frequency'})
    #add dashed line to indicate the accuracy score of the model line y location is maximum count of histogram
    fig.add_shape(type='line', yref='paper', y0=0, y1=1, xref='x', x0=acc_score, x1=acc_score, line=dict(dash='dash', color='red', width=3))


    fig.add_annotation(dict(font=dict(color="black",size=14),
                            #x=x_loc,
                            x=0,
                            y=1.25,
                            #y=1.18,
                            showarrow=False,
                            text='Number of permutation: {}'.format(n_permutate),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")

    fig.add_annotation(dict(font=dict(color="black",size=14),
                            #x=x_loc,
                            x=0,
                            y=1.18,
                            showarrow=False,
                            text='Accuracy score: {}'.format(np.round(acc_score, decimals=3)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            #x=x_loc,
                            x=0,
                            y=1.11,
                            showarrow=False,
                            text='<i>p-value</i>: {}'.format(np.round(p_value, decimals=6)),
                            textangle=0,
                            xref="paper",
                            yref="paper"),
                            # set alignment of text to left side of entry
                            align="left")

    fig.update_layout(showlegend=False)

    fig.update_layout(title_x=0.5)

    #fig.show()
    fig.write_image("{}/hist_plot/Permutation_scores_{}.png".format(path_save, name))
    fig.write_html("{}/hist_plot/Permutation_scores_{}.html".format(path_save, name))
    
    
    
    #S plot
    # sub-plot covariance for x and correlation for y S-plot using plotly, color by covariance with jet colormap
    #setup figure size


    fig = px.scatter(s_scores_df, x='covariance', y='correlation', color='covariance', range_color=[-1,1],
                     color_continuous_scale='jet', text=s_scores_df.index, height=900, width=2000)
    fig.update_layout(title='<b>S-plot</b>', xaxis_title='Covariance', yaxis_title='Correlation')

    #add line of axis and set color to black and line width to 2 pixel
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    #Add tick width to 2 pixel
    fig.update_xaxes(tickwidth=2)
    fig.update_yaxes(tickwidth=2)
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
    fig.update_yaxes(tickformat=",.0")
    #fig.update_xaxes(tickformat=",.0")
    fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_layout(
        title={
            'y':1,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        font=dict(size=20))
    #Set font size to 20
    #Set marker size to 5 pixel
    fig.update_traces(marker=dict(size=14))
    #fig.show()
    fig.write_image("{}/s_plot/S_plot_{}.png".format(path_save, name))
    fig.write_html("{}/s_plot/S_plot_{}.html".format(path_save, name))
    

    #Loadings plot
    
    # X * 1 when correlation is positive, X * -1 when correlation is negative
    def median_corr(X):
        X_corr = np.median(X, axis=0)
        X_corr = X_corr * np.sign(s_scores_df['correlation'])
        return X_corr

    X2 = median_corr(X)

    fig = px.scatter(s_scores_df, x=ppm, y=X2, color='covariance', color_continuous_scale='jet', text=s_scores_df.index, height=500, width=2000)

    fig.update_traces(marker=dict(size=3))
    fig.update_xaxes(autorange="reversed")
    fig.update_layout(title='<b>Median spectra</b>', xaxis_title='ppm', yaxis_title='Correlation')
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
    #Add tick width to 2 pixel
    fig.update_xaxes(tickwidth=2)
    fig.update_yaxes(tickwidth=2)

    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
    fig.update_yaxes(tickformat=",.0")
    #fig.update_xaxes(tickformat=",.0")
    fig.update_layout(
        title={
            'y':1,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        font=dict(size=20))
    #Set marker size to 5 pixel
    fig.update_traces(marker=dict(size=3))
    #fig.show()
    fig.write_image("{}/loading_plot/loadings_plot_{}.png".format(path_save, name))
    fig.write_html("{}/loading_plot/loadings_plot_{}.html".format(path_save, name))
    
    from lingress.Lingress import lin_regression
    lin_mod = lin_regression(x=df_X, target=meta[Group], label=meta[Group], features_name=ppm)
    lin_mod.create_dataset()
    lin_mod.fit_model(adj_method='fdr_bh')
    report = lin_mod.report()
    report.to_csv("{}/Lingress/lingress_report_{}.csv".format(path_save, name))
    lin_mod.volcano_plot()
    lin_mod.png_plot(plot_name="lingress/volcano_plot_{}".format(name), path_save=path_save)
    lin_mod.html_plot(plot_name="lingress/volcano_plot_{}".format(name), path_save=path_save)

    del X, Y, Y1, ppm, spectra, meta, df_X, s_scores_df, df_opls_scores, cv, oplsda, pipeline, acc_score, permutation_scores, p_value, fig, lin_mod, report, n_permutate, plot_name, 
    
    print("Finish {}".format(name))



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1826172344592381s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.12208771705627441s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.19360089302062988s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    2.3s
[Parallel(n_jobs=

adjustment p-value with Benjamini/Hochberg (non-negative) Done
Finish sham ad libitum, 2-wk post-op_vs_CR + INT777 (H), 2-wk post-op


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0976099967956543s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.16067719459533691s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.17542314529418945s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 384 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:    1.8s
[Parallel(n_jobs=

adjustment p-value with Benjamini/Hochberg (non-negative) Done
Finish sham ad libitum, 2-wk post-op_vs_CR + INT777 (L), 2-wk post-op


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.05982208251953125s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.09436583518981934s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.16278910636901855s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 384 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:    1.7s
[Parallel(n_jobs

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0641021728515625s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1078488826751709s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1715068817138672s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 384 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1

adjustment p-value with Benjamini/Hochberg (non-negative) Done
Finish CR + INT777 (L), 2-wk post-op_vs_CR + INT777 (H), 1-wk pre-op


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.14345908164978027s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.19934517919079425s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 148 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.19651234626770023s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.5s
[Parallel(n_jobs

adjustment p-value with Benjamini/Hochberg (non-negative) Done
Finish sham ad libitum, 1-wk post-op_vs_CR + INT777 (H), 1-wk post-op


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.07113504409790039s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.16689801216125488s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 308 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 436 tasks      | elaps